In [1]:
import sys

# Print the version of Python
print(sys.version)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [2]:
!pip -q install langchain==0.0.330 openai
!pip -q install duckduckgo-search
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:0

Setting up some keys

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.330
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 



# Custom Tools & Agents 🤖

In [4]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('.env')

# Retrieve the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key is not None and len(openai_api_key) != 0:
    print("API key retrieved successfully.")
else:
    print("API key not found.")

API key retrieved successfully.


In [5]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [40]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k-0613")

## Standard Tool

In [83]:
from langchain.tools import DuckDuckGoSearchRun, BaseTool
from langchain.agents import initialize_agent, AgentType, Tool, AgentExecutor
from langchain.document_loaders import RecursiveUrlLoader
from langchain.chains import LLMMathChain
from bs4 import BeautifulSoup as bs
import re
import requests

search = DuckDuckGoSearchRun()
llm_math_chain = LLMMathChain.from_llm(llm=turbo_llm, verbose=True)

In [43]:
# class WebPageTool(BaseTool):
#     name = "Get Webpage"
#     description = "Useful for when you need to get the content from a specific webpage"

#     def _run(self, webpage: str):
#         response = requests.get(webpage)
#         html_content = response.text

#         def strip_html_tags(html_content):
#             soup = bs(html_content, "html.parser")
#             stripped_text = soup.get_text()
#             return stripped_text

#         stripped_content = strip_html_tags(html_content)
#         if len(stripped_content) > 4000:
#             stripped_content = stripped_content[:4000]
#         return stripped_content

#     def _arun(self, webpage: str):
#         raise NotImplementedError("This tool does not support async")

# page_getter = WebPageTool()

def extract_text_and_limit_tokens(html, token_limit=4097):
    # Extract text with BeautifulSoup
    text = bs(html, "html.parser").text
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text.strip())

    # Approximate tokenization by splitting on spaces
    tokens = text.split(' ')

    # Limit the number of tokens and join back into a string
    limited_text = ' '.join(tokens[:token_limit])
    return limited_text

def crawl_site(url):
    loader = RecursiveUrlLoader(
        url=url,
        max_depth=2,
        extractor=lambda x: extract_text_and_limit_tokens(x, 4097)
    )
    docs = loader.load()
    return docs

In [68]:
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="Search the internet to find helpful websites."
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="Useful for when you need to answer questions about math",
    ),
    Tool(
        name="site_crawler",
        func=crawl_site,
        description="Crawl a website up to depth 2."
    ),
]

mrkl = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    # max_iterations=3,
    early_stopping_method='generate',
)

In [69]:
# search.run('Weather in Atlanta, GA today')

In [79]:
fixed_prompt = '''You are a helpful AI assistant.''' # can change this later
mrkl.agent.prompt.messages[0].content = fixed_prompt
mrkl.agent.prompt.messages[0].content

'You are a helpful AI assistant.'

In [80]:
result = mrkl.run("Give me a list of at least 5 good restaurants around Atlanta, GA. Include what people usually eat, and what is the estimated price in number. Research in depth for each restaurants by visting their restaurants websites")



> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'good restaurants in Atlanta, GA'}`


Head to the bar for cocktails, a glass of wine, and the a la carte menu. Open in Google Maps. Foursquare. 1460 Ellsworth Industrial Blvd NW, Atlanta, GA 30318. (404) 365-0410. Visit Website. Ryan Fleisher. Also featured in: 16 Atlanta Restaurants Worth the Splurge for a Special Occasion. 15. The Colonnade. The Colonnade has been serving meat-and-three plates to hungry locals and visitors in some form and location for almost a century (it opened in 1927). It is not a fancy or ... Atlas, Mujo, Bacchanalia, Hayakawa, and Lazy Betty are Atlanta's One Star Michelin restaurants for 2023. Ryan Fleisher. Beth McKibben is the editor and staff reporter for Eater Atlanta and has been covering food and cocktails locally and regionally for 12 years. The first Atlanta Michelin guide just dropped, with Atlas and Mujo counted among ... There's something for everyone here: wood-fired pizzas

In [82]:
print(result)

I apologize, but I couldn't find a list of good restaurants in Atlanta, GA with in-depth information about each restaurant. However, I can provide you with a list of popular restaurants in Atlanta along with some general information about them:

1. Bacchanalia: Located at 1460 Ellsworth Industrial Blvd NW, Bacchanalia is a highly acclaimed restaurant known for its fine dining experience. The menu features seasonal and locally sourced ingredients, and the restaurant offers a prix fixe menu. The estimated price range is high.

2. Gunshow: Situated at 924 Garrett Street, Gunshow offers a unique dining experience where chefs present their dishes dim sum-style to diners. The menu is constantly changing and features a variety of creative and innovative dishes. The estimated price range is moderate to high.

3. Empire State South: Located at 999 Peachtree Street, Empire State South is a popular restaurant known for its modern Southern cuisine. The menu showcases dishes made with locally sourc